In [ ]:
# Import necessary packages
import nltk
# nltk.download('wordnet')
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import pickle
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize NLTK's WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Sample dataset of intents and responses
intents = {
    "greeting": {"patterns": ["hello", "hi", "hey"], "responses": ["Hello!", "Hi there!", "Hey!"]},
    "goodbye": {"patterns": ["bye", "farewell"], "responses": ["Goodbye!", "See you later!", "Bye!"]},
    "thanks": {"patterns": ["thank you", "thanks"], "responses": ["You're welcome!", "Anytime!", "Glad I could help!"]}
}

# Tokenize and lemmatize the patterns
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

# Preprocess the patterns and create training data
training_data = []
output_labels = []
for intent, data in intents.items():
    for pattern in data["patterns"]:
        tokens = preprocess_text(pattern)
        training_data.append(" ".join(tokens))
        output_labels.append(intent)

# Vectorize the training data
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(training_data)

# Train a simple classifier (here we're using cosine similarity)
classifier = cosine_similarity(X)

# Save the classifier and vectorizer as pickle files
with open("classifier.pkl", "wb") as f:
    pickle.dump(classifier, f)
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

# Function to get response from the chatbot
def get_response(text):
    # Preprocess input text
    tokens = preprocess_text(text)
    input_vector = vectorizer.transform([" ".join(tokens)])
    # Calculate similarity between input and training data
    similarities = cosine_similarity(input_vector, X)
    # Get index of highest similarity
    index = np.argmax(similarities)
    # Get corresponding intent label
    intent_label = output_labels[index]
    # Choose a random response from intents
    response = random.choice(intents[intent_label]["responses"])
    return response

# Test the chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    response = get_response(user_input)
    print("Bot:", response)


You: hi
Bot: Hi there!
You: bye
Bot: Bye!
